In [5]:
import pandas as pd
import numpy as np
from benchmarks import *
import glob
import os, sys
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
from tqdm.notebook import tqdm

In [6]:
MACHINE = 'lassen' if 'lassen' in ROOT_DIR else 'ruby'
print(MACHINE, ROOT_DIR)
prognames = list(progs.keys())
probsizes = ['smlprob', 'medprob', 'lrgprob']

logsDir = ROOT_DIR+'/logs'

goMethods=['cma', 'pso', 'bo']
seeds = [1337, 3827, 9999, 4873]

lassen /usr/WS2/bolet1/lassen-benchmarks/exploreGlobalOptimizations


In [7]:
def getHypersFromFilename(filename):

	# cut the path out if it has a path, and remove the .csv extension
	filename = os.path.splitext(os.path.basename(filename))[0]

	if '-BO-' in filename:
		if '-ucb-' in filename:
			filename = filename[filename.find('-ucb-')+5:]
			found = re.findall(r'(?:k)(.*)(?:-kd)(.*)(?:-kdd)(.*)(?:--DONE)',filename)[0]
			k = float(found[0])
			kd = float(found[1])
			kdd = int(found[2])
			return {'utilFnct':'ucb', 'kappa':k, 'kappa_decay':kd, 'kappa_decay_delay':kdd}
		
		elif '-ei-' in filename:
			filename = filename[filename.find('-ei-')+4:]
			found = re.findall(r'(?:xi)(.*)(?:--DONE)',filename)[0]
			xi = float(found[0])
			return {'utilFnct':'ei', 'xi':xi}
		
		elif '-poi-' in filename:
			filename = filename[filename.find('-poi-')+5:]
			found = re.findall(r'(?:xi)(.*)(?:--DONE)',filename)[0]
			xi = float(found[0])
			return {'utilFnct':'poi', 'xi':xi}

	elif '-PSO-' in filename:
			filename = filename[filename.find('-PSO-')+5:]
			found = re.findall(r'(?:pop)(.*)(?:-w)(.*)(?:-c1)(.*)(?:-c2)(.*)(?:--DONE)',filename)[0]
			pop = int(found[0])
			w = float(found[1])
			c1 = float(found[2])
			c2 = float(found[3])
			return {'popsize':pop, 'w':w, 'c1':c1, 'c2':c2}
	
	elif '-CMA-' in filename:
			filename = filename[filename.find('-CMA-')+5:]
			found = re.findall(r'(?:sigma)(.*)(?:-pop)(.*)(?:-popdecay)(.*)(?:--DONE)',filename)[0]
			sigma = float(found[0])
			pop = int(found[1])
			pd = float(found[2])
			#return {'method':'cma', 'sigma':sigma, 'popsize':pop, 'popdecay':pd}
			# excluded popdecay in experiments
			return {'sigma':sigma, 'popsize':pop}
	
	else:
		print('no GO method in filename...')
		raise ValueError('Failed to provide a string path with a GO Method')
		return


In [8]:
def getData(progname, probsize, method, seed):
	mainDF = pd.DataFrame(columns=['progname', 'probsize', 'method', 'seed'])
	startTime = time.time()

	dir = logsDir+f'/{progname}-{probsize}/{method}-{seed}'
	print('working on', dir)
	# check that the directory exists
	# grab the completed data
	csvs = glob.glob(dir+'/*DONE.csv')

	# read in each of the dataframe, then contatenate them all at once
	readin = []
	for csv in tqdm(csvs):
		# extract the hyperparam values from the filename
		hypers = getHypersFromFilename(csv)
		toadd = pd.read_csv(csv, usecols=['globalSample', 'optimXtime', 'xtime'])

		# add the extra columns
		for hyper,val in hypers.items():
			toadd[hyper] = val

		readin += [toadd]

	mainDF = pd.concat([mainDF]+readin, ignore_index=True)

	mainDF['progname'] = progname
	mainDF['probsize'] = probsize
	mainDF['method'] = method
	mainDF['seed'] = seed 

	print('completed', progname, probsize, method, seed, mainDF.shape, (time.time() - startTime), 'secs')		
	return mainDF

In [9]:
dbFile = f'{MACHINE}-fullExploreDataset.csv'
xtimeDB = pd.read_csv(ROOT_DIR+'/databases/'+dbFile)

def getMinMaxXtimeForProg(progname, probsize):
	df = xtimeDB[(xtimeDB['progname'] == progname) & 
							 (xtimeDB['probsize'] == probsize)]['xtime']
	return (df.min(), df.max())

# 0 will mean close to maxXtime
# 1 will mean close to minXtime
def convertXtimesToPercent(df, minXtime, maxXtime):
	df['xtime'] = 1 - (df['xtime']-minXtime)/(maxXtime-minXtime)
	return


In [10]:
def preprocessAllLogs():

	for method in goMethods:
		for progname in prognames:
			# gather all the data
			toJoin = []
			for probsize in probsizes:
				minXtime, maxXtime = getMinMaxXtimeForProg(progname, probsize)
				for seed in seeds:
					df = getData(progname, probsize, method, seed)
					convertXtimesToPercent(df, minXtime, maxXtime)
					toJoin += [df]

			fullDF = pd.concat(toJoin, ignore_index=True)

			# let's save the dataframe for future re-use
			filename = ROOT_DIR+'/databases/'+f'{MACHINE}-{progname}-{method}-GO_Data.csv'
			print('\n\n wrote:', filename, '\n\n')
			fullDF.to_csv(filename, index=False)
	return

In [11]:
preprocessAllLogs()

working on /usr/WS2/bolet1/lassen-benchmarks/exploreGlobalOptimizations/logs/bt_nas-smlprob/cma-1337


  0%|          | 0/100 [00:00<?, ?it/s]

completed bt_nas smlprob cma 1337 (59400, 9) 0.46275901794433594 secs
working on /usr/WS2/bolet1/lassen-benchmarks/exploreGlobalOptimizations/logs/bt_nas-smlprob/cma-3827


  0%|          | 0/100 [00:00<?, ?it/s]

completed bt_nas smlprob cma 3827 (59400, 9) 0.41424059867858887 secs
working on /usr/WS2/bolet1/lassen-benchmarks/exploreGlobalOptimizations/logs/bt_nas-smlprob/cma-9999


...

completed lulesh lrgprob bo 3827 (1629342, 12) 14.312270879745483 secs
working on /usr/WS2/bolet1/lassen-benchmarks/exploreGlobalOptimizations/logs/lulesh-lrgprob/bo-9999


  0%|          | 0/2573 [00:00<?, ?it/s]

completed lulesh lrgprob bo 9999 (1528362, 12) 13.934394359588623 secs
working on /usr/WS2/bolet1/lassen-benchmarks/exploreGlobalOptimizations/logs/lulesh-lrgprob/bo-4873


  0%|          | 0/2577 [00:00<?, ?it/s]

completed lulesh lrgprob bo 4873 (1530738, 12) 13.4993155002594 secs


 wrote: /usr/WS2/bolet1/lassen-benchmarks/exploreGlobalOptimizations/databases/lassen-lulesh-bo-GO_Data.csv 


